In [4]:
import os
import pandas as pd

# Корневая папка с датасетом (текущая папка, где будет ноутбук)
dataset_dir = os.getcwd()

# Список для хранения данных
data = []

# Проходим по всем подпапкам
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".wav"):
            wav_path = os.path.join(root, file)
            txt_file = file.replace(".wav", ".txt")
            txt_path = os.path.join(root, txt_file)

            # Проверяем, существует ли текстовый файл
            if os.path.exists(txt_path):
                with open(txt_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                
                # Формируем путь относительно Dataset и с прямыми слешами
                rel_path = os.path.relpath(wav_path, dataset_dir).replace("\\", "/")
                
                data.append({
                    "path": rel_path,
                    "text": text
                })

# Создаем DataFrame и сохраняем в CSV
df = pd.DataFrame(data)

df['text'] = df['text'].str.replace(r'[^A-Za-zА-Яа-яЁё\s]', '', regex=True)

df['text'] = df['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

df['text'] = df['text'].str.lower()

df["path"] = "Dataset/" + df["path"]

csv_path = os.path.join(dataset_dir, "git_dataset.csv")
df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"CSV файл создан: {csv_path}")


CSV файл создан: e:\Education\4 course 1 semester\Course project\Shards_prodject\Code\Task\Dataset\git_dataset.csv


In [6]:
import pandas as pd
import glob
import os
import re

# Папка с вашими .tsv файлами
folder_path = r"E:\Education\4 course 1 semester\Course project\Shards_prodject\Code\Task\Dataset\cv-corpus-23.0-2025-09-05"


# Получаем список всех .tsv файлов
tsv_files = glob.glob(os.path.join(folder_path, "**", "*.tsv"))
print(tsv_files)
# Список для хранения датафреймов
dataframes = []

for file in tsv_files:
    try:
        # Читаем файл
        df = pd.read_csv(file, sep="\t")

        # Проверяем, есть ли колонки 'path' и 'text'
        if 'path' in df.columns and 'sentence' in df.columns:
            # Берем только нужные колонки
            df = df[['path', 'sentence']].rename(columns={'sentence': 'text'})
            dataframes.append(df)
        else:
            print(f"Пропущен файл {file}, нет нужных колонок")

    except Exception as e:
        print(f"Ошибка при чтении файла {file}: {e}")

# Объединяем все датафреймы в один
if dataframes:
    moz_df = pd.concat(dataframes, ignore_index=True)
    # Оставляем только буквы и пробелы
    moz_df['text'] = moz_df['text'].str.replace(r'[^A-Za-zА-Яа-яЁё\s]', '', regex=True)

    # Убираем лишние пробелы
    moz_df['text'] = moz_df['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

    # Приводим к нижнему регистру
    moz_df['text'] = moz_df['text'].str.lower()
    # Сохраняем результат

    moz_df["path"] = "Dataset/cv-corpus-23.0-2025-09-05/ru/clips" + moz_df["path"]

    moz_df.to_csv("moz_dataset.csv", index=False)
    print("Датасет успешно создан: maz_dataset.csv")
else:
    print("Нет файлов с нужными колонками")


['E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\clip_durations.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\dev.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\invalidated.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\other.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\reported.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\test.tsv', 'E:\\Education\\4 course 1 semester\\Course project\\Shards_prodject\\Code\\Task\\Dataset\\cv-corpus-23.0-2025-09-05\\ru\\train.tsv', 'E:\\Education\\4 course 1 semester\\Course pr

C:\Users\follo\AppData\Local\Temp\ipykernel_4304\2977180630.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


Пропущен файл E:\Education\4 course 1 semester\Course project\Shards_prodject\Code\Task\Dataset\cv-corpus-23.0-2025-09-05\ru\validated_sentences.tsv, нет нужных колонок
Датасет успешно создан: maz_dataset.csv


In [13]:
df1 = pd.read_csv('E:/Education/4 course 1 semester/Course project/Shards_prodject/Code/Task/Dataset/git_dataset.csv')
df2 = pd.read_csv('E:/Education/4 course 1 semester/Course project/Shards_prodject/Code/Task/Dataset/moz_dataset.csv')

final_df = pd.concat([df1, df2], ignore_index=True)

final_df.to_csv('E:/Education/4 course 1 semester/Course project/Shards_prodject/Code/Task/Dataset/final_dataset.csv', index=False)

In [14]:
!pip install pandas
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("final_dataset.csv")

train_df, temp_df = train_test_split(
    df,
    test_size=0.2,       # 20% уйдёт на val+test
    random_state=42,
    shuffle=True
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,       # половина temp → test, половина → val
    random_state=42,
    shuffle=True
)

print("Train:", len(train_df))
print("Validation:", len(val_df))
print("Test:", len(test_df))

# 4️⃣ При желании сохраняем в CSV
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

Train: 220901
Validation: 27613
Test: 27613
